In [1]:
import pymongo

In [2]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017')

In [3]:
db = client.flaskdb

In [ ]:
list(db.ingredients.find())


In [5]:
import yaml
import pathlib

with pathlib.Path("config.yml").open("r") as f:
    ingredients = yaml.load(f, Loader=yaml.FullLoader)

db.ingredients.delete_many({})

for name, cat in ingredients["varer"].items():
    db.ingredients.insert_one(
        {
            "name":name,
            "placement":cat,
        }
    )


In [6]:
db.recipes.delete_many({})

for recipe in pathlib.Path("recipes/").glob("*.yml"):
    with recipe.open("r") as f:
        recipe = yaml.load(f, Loader=yaml.FullLoader)
        db.recipes.insert_one(recipe)
recipe



{'navn': 'Grove pølser med æblesaute og mos',
 'placering': 'Tung mad / gris',
 'antal': 4,
 'ingredienser': {'grove pølser': {'amount': 6, 'unit': 'stk'},
  'æbler': {'amount': 2, 'unit': 'stk'},
  'rød peberfrugt': {'amount': 2, 'unit': 'stk'},
  'porre': {'amount': 2, 'unit': 'stk'},
  'hvidløg': {'amount': 2, 'unit': 'fed'},
  'tørret timian': {'amount': 2, 'unit': 'tsk'},
  'rapsolie': {'amount': 10, 'unit': 'ml'},
  'kartofler': {'amount': 600, 'unit': 'g'},
  'hønsebouillon': {'amount': 3, 'unit': 'stk'},
  'sennep': {'amount': 1, 'unit': 'dl'}},
 '_id': ObjectId('609e6c5c7316c730384a965a')}

In [7]:
db.menus.delete_many({})

def flatten(d):
    try:
        for k, v in d.items():
            yield {"name":k, **v}
    except:
        raise

for menu_path in pathlib.Path("menus/").glob("*.yaml"):
    try:
        _, w, y = menu_path.stem.split("_")
    except:
        continue
    w = int(w)
    y = int(y)
    
    with menu_path.open("r") as f:
        menu = yaml.load(f, Loader=yaml.FullLoader)
        
        menu_dict = {
            "week":w,
            "year":y,
            "dishes":[
            {
                "title": title,
                "days": [],
                "components": [list(flatten(d))[0] for d in definition],
            } for title, definition in menu.items()
            ],
        }

        db.menus.insert_one(menu_dict)


In [16]:
menu = db.menus.find_one({"week":6})

import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(menu.keys())

print(f" week {menu['week']} {menu['year']} ")



dict_keys(['_id', 'week', 'year', 'dishes'])
 week 6 2021 
